## Deliverable 2. Create a Customer Travel Destinations Map.

In [13]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from vacation_search_config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [14]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,70.09,67,100,26.28,overcast clouds
1,1,Saint-Augustin,CA,51.2260,-58.6502,62.31,66,21,2.35,few clouds
2,2,Samarai,PG,-10.6167,150.6667,80.67,72,77,14.34,broken clouds
3,3,Hilo,US,19.7297,-155.0900,73.26,94,94,5.75,light rain
4,4,Jamestown,US,42.0970,-79.2353,65.01,94,1,1.99,clear sky


In [15]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


In [16]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Samarai,PG,-10.6167,150.6667,80.67,72,77,14.34,broken clouds
6,6,Kapaa,US,22.0752,-159.3190,80.92,78,90,5.01,light rain
7,7,Hithadhoo,MV,-0.6000,73.0833,81.82,68,57,0.81,broken clouds
19,19,Sahbuz,AZ,39.4072,45.5739,80.20,35,6,3.62,clear sky
20,20,San Patricio,US,28.0170,-97.5169,84.09,81,1,7.83,clear sky
24,24,Namtsy,RU,62.7161,129.6658,82.51,32,29,7.70,scattered clouds
30,30,Luganville,VU,-15.5333,167.1667,81.36,74,25,9.26,scattered clouds
34,34,Kavaratti,IN,10.5669,72.6420,82.38,79,100,17.92,overcast clouds
40,40,Eyl,SO,7.9803,49.8164,80.40,62,89,25.75,overcast clouds
43,43,Butaritari,KI,3.0707,172.7902,83.95,72,100,10.96,overcast clouds


In [17]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

clean_df = preferred_cities_df.dropna()


In [18]:
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Samarai,PG,-10.6167,150.6667,80.67,72,77,14.34,broken clouds
6,6,Kapaa,US,22.0752,-159.3190,80.92,78,90,5.01,light rain
7,7,Hithadhoo,MV,-0.6000,73.0833,81.82,68,57,0.81,broken clouds
19,19,Sahbuz,AZ,39.4072,45.5739,80.20,35,6,3.62,clear sky
20,20,San Patricio,US,28.0170,-97.5169,84.09,81,1,7.83,clear sky
...,...,...,...,...,...,...,...,...,...,...
698,698,Dhidhdhoo,MV,6.8833,73.1000,83.34,73,96,11.10,overcast clouds
703,703,Bada,RU,51.3920,109.8717,81.45,49,83,1.10,light rain
705,705,Snyder,US,32.7179,-100.9176,80.94,65,1,4.61,clear sky
713,713,Fereydun Kenar,IR,36.6864,52.5225,84.13,66,4,0.96,clear sky


In [19]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Samarai,PG,80.67,broken clouds,-10.6167,150.6667,
6,Kapaa,US,80.92,light rain,22.0752,-159.3190,
7,Hithadhoo,MV,81.82,broken clouds,-0.6000,73.0833,
19,Sahbuz,AZ,80.20,clear sky,39.4072,45.5739,
20,San Patricio,US,84.09,clear sky,28.0170,-97.5169,
24,Namtsy,RU,82.51,scattered clouds,62.7161,129.6658,
30,Luganville,VU,81.36,scattered clouds,-15.5333,167.1667,
34,Kavaratti,IN,82.38,overcast clouds,10.5669,72.6420,
40,Eyl,SO,80.40,overcast clouds,7.9803,49.8164,
43,Butaritari,KI,83.95,overcast clouds,3.0707,172.7902,


In [20]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}


# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame

    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
   
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [21]:
# 7. Drop the rows where there is no Hotel Name.

clean_hotel_df = hotel_df.dropna()

In [22]:
# 8a. Create the output File (CSV)

output_data_file = "WeatherPy_Vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [23]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Conditions</dt><dd>{Current Description} & {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [24]:
# 11a. Add a marker layer for each city to the map. 

locations = hotel_df[["Lat", "Lng"]]

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)

# 11b. Display the figure

fig


Figure(layout=FigureLayout(height='420px'))